# 01 Instalação e Importação das Bibliotecas

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import RandomOverSampler

# 02 Importação do conjunto de dados

In [ ]:
# autenticação BQ
from google.colab import auth

auth.authenticate_user()

In [ ]:
# autenticação drive
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project='Insira o nome do seu projeto aqui')

train_data = client.query('''SELECT * FROM `nme_projeto.youtube.train_data`''').to_dataframe()

# 03 Processamento dos dados e Modelo Estatístico

In [ ]:
documents = train_data['textStemmed'].values

# crio o objeto para a conversão da coleção de documentos brutos em uma matriz de recursos do TF-IDF.
# o texto que será processado é o lapidado, ou seja, a variavel textStemmed do arquivo.
# as variaveis do objeto TfidfVectorizer precisam ser identicas as usadas no programa '01.b. Tratamento dos Dados'
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=1000, min_df=5, max_df=0.7).fit(documents)

In [ ]:
# aplico o objeto na lista de documentos e armazeno a informacao na variavel preditora X
X = vectorizer.transform(documents)

# armazeno a informacao de cluster na variavel de resposta Y
y = train_data.cluster.values

# crio o objeto para realizar sobreamostragem aleatória
oversample = RandomOverSampler(random_state=42)

# faz uma nova amostra do conjunto de dados.
X_over, y_over = oversample.fit_resample(X, y)

# divisao aleatória da amostra em subconjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.3, random_state=42)

# crio um objeto classificador de regressão logística
# uso o algoritmo de treinamento com esquema one-vs-rest (OvR), habilitando a opção 'multi_class' como 'ovr'.
classifier = LogisticRegression(
                                multi_class='ovr',
                                solver='lbfgs',
                                class_weight='balanced', # usa os valores de y para ajustar automaticamente os 
                                                         # pesos inversamente proporcionais às frequências da classe nos dados de entrada 
                                random_state=42
                                ).fit(X_train, y_train)

# 04 Avaliação do Modelo

Com a matriz de confusão é possível tirar algumas métricas:
*   Precision: Entre a classe positivo, quantos o modelo acertou.
*   Recall: Entre a classe positiva com valor esperado, quantas estão certas.
*   F1-score: Média harmônica entre Precision e Recall.

In [ ]:
y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))

                                          precision    recall  f1-score   support

                              jogo é lua       1.00      1.00      1.00        45
                         mudança de vida       1.00      1.00      1.00        42
otimismo com subida/valorização da moeda       1.00      1.00      1.00        48
                   querem entrar no jogo       1.00      1.00      1.00        49
                    sobre mercado de NFT       1.00      0.98      0.99        48
                      sobre o jogo em si       1.00      1.00      1.00        43
      sobre o momento de entrada no jogo       0.97      1.00      0.99        37
                sobre perfomance no jogo       1.00      1.00      1.00        44

                                accuracy                           1.00       356
                               macro avg       1.00      1.00      1.00       356
                            weighted avg       1.00      1.00      1.00       356



# Aplicação do Modelo a nova base (df_comments)

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project='Insira o nome do seu projeto aqui')

df_comments = client.query('''SELECT * FROM `nme_projeto.youtube.df_comments`''').to_dataframe()

In [ ]:
import numpy as np
import pandas as pd

def define_cluster(sentence):
  '''
  Input:
      sentence: frase a ser classificada
  Output:
      cluster: cluster de maior probabilidade da frase estar relacionada
  '''

  document = [sentence]

  # transformo a frase em vetores
  document = vectorizer.transform(document)

  # calculo a probabilidade da frase pertencer a cada um dos possiveis clusters
  proba = classifier.predict_proba(document)

  # identifico qual o indice da maior probabilidade
  n = np.argmax(proba, axis=1)[0]

  if proba[0][n] > 0.7:
    # identifico qual o nome do cluster de maior probabilidade
    cluster = classifier.classes_[n]
  else:
    cluster = 'desconhecido'

  return cluster

df_comments['cluster'] = df_comments.textStemmed.apply(lambda x: define_cluster(x))

df_comments = df_comments[df_comments['cluster'] != 'desconhecido']

In [ ]:
print("Base de treino possui %s registros"%train_data.shape[0])

# incluo os novos valores classificados, junto a base de treino (de modo a enriquece-la cada vez mais)
nova_base = pd.concat([train_data,df_comments])

nova_base.drop_duplicates(inplace=True)

# Cria um dataframe de back-up
last_train_data = train_data

print("Nova base de treino (enriquecida) possui %s registros"%nova_base.shape[0])

Base de treino possui 748 registros
Nova base de treino (enriquecida) possui 748 registros


In [ ]:
import pandas_gbq

from google.oauth2 import service_account

train_data = nova_base

# Armazena no GCP a nova base treinada (com incremento de informações)
def save_gbq(df, channel_name, nome_tabela, path_json_key):
  '''
  Input:
      df: conjunto de dados a ser carregado no GCP
      channel_name: nome do canal
      nome_tabela: nome que será dado ao conjunto de dados carregado
      path_json_key: chave json para a autenticaçao do acesso
  '''
  credentials = service_account.Credentials.from_service_account_file(path_json_key)
  pandas_gbq.to_gbq(dataframe = df,
                    destination_table = f'{channel_name}.{nome_tabela}',
                    project_id = 'Insira o nome do seu projeto aqui',
                    credentials = credentials,
                    if_exists='replace')

save_gbq(train_data, 'youtube', 'train_data', 'Insira o caminho de seu arquivo json aqui')

# Armazena no GCP a última base treinada (antes do incremento de novas informações)
def save_gbq(df, channel_name, nome_tabela, path_json_key):
  '''
  Input:
      df: conjunto de dados a ser carregado no GCP
      channel_name: nome do canal
      nome_tabela: nome que será dado ao conjunto de dados carregado
      path_json_key: chave json para a autenticaçao do acesso
  '''
  credentials = service_account.Credentials.from_service_account_file(path_json_key)
  pandas_gbq.to_gbq(dataframe = df,
                    destination_table = f'{channel_name}.{nome_tabela}',
                    project_id = 'Insira o nome do seu projeto aqui',
                    credentials = credentials,
                    if_exists='replace')

save_gbq(last_train_data, 'youtube', 'last_train_data', 'Insira o caminho de seu arquivo json aqui')

1it [00:04,  4.39s/it]
1it [00:03,  3.29s/it]
